# Model testing

- It's just a testing ground for the model. One model is loaded and a `cmc.test_model` is applied on it.
- As a result confusion matrix, ROC, AUROC, F1, PR curves will be generated and saved to the model's folder

In [1]:
import os
from pathlib import Path
import confinement_mode_classifier as cmc
import torchvision
import torch
from tqdm import tqdm
import torch.nn as nn
import re
import pandas as pd
path = Path(os.getcwd())
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

KeyboardInterrupt: 

In [ ]:
ris_option = 'RIS1'
second_img_opt = 'RIS2'
num_workers = 32
num_epochs_for_fc = 10
num_epochs_for_all_layers = 10
batch_size = 16
learning_rate_min = 0.001
learning_rate_max = 0.01
comment_for_model_name = ris_option + 'x' + second_img_opt  + f'{...}'

In [ ]:
shot_usage = pd.read_csv(f'{path}/data/shot_usage.csv')
shot_for_ris = shot_usage[shot_usage['used_for_ris1'] & shot_usage['used_for_ris2']]
shot_numbers = shot_for_ris['shot']
shots_for_testing = shot_for_ris[shot_for_ris['used_as'] == 'test']['shot']
shots_for_validation = shot_for_ris[shot_for_ris['used_as'] == 'val']['shot']


shot_df, test_df, val_df, train_df = cmc.load_and_split_dataframes(path, shot_numbers, 
                                                                   shots_for_testing, 
                                                                   shots_for_validation, use_ELMS=False)

#Get dataloaders. second_img_opt='RIS1' indicates that two RIS1 models will be ensembled
test_dataloader = cmc.get_dloader(test_df, path=path, batch_size=batch_size,
                                   shuffle=False, balance_data=True, 
                                   ris_option=ris_option, second_img_opt=second_img_opt, 
                                   num_workers=num_workers)

val_dataloader = cmc.get_dloader(val_df, path=path, batch_size=batch_size,
                                   shuffle=False, balance_data=True, 
                                   ris_option=ris_option, second_img_opt=second_img_opt, 
                                   num_workers=num_workers)

train_dataloader = cmc.get_dloader(train_df, path=path, batch_size=batch_size,
                                   shuffle=False, balance_data=True, 
                                   ris_option=ris_option, second_img_opt=second_img_opt, 
                                   num_workers=num_workers)

dataloaders = {'train':train_dataloader, 'val':val_dataloader}
dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'val']}

### Load the model

In [ ]:
#Initiate a single camera model
pretrained_model = torchvision.models.resnet18(weights='IMAGENET1K_V1', )
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 3) #3 classes: L-mode, H-mode, ELM
pretrained_model = pretrained_model.to(device)

#Which model to test?
model_path = f'{path}/vyzkumny_ukol/runs/24-02-23, 16-15-22 RIS1xRIS2Ellipsis_classifier_training'

#TEST one camera model
# pretrained_model.load_state_dict(torch.load(f'{model_path}/model.pt'))
# pretrained_model.eval()
# pretrained_model.to(device)

#TEST ensembled model
ensembled_model = cmc.TwoImagesModel(modelA=pretrained_model, modelB=pretrained_model, hidden_units=30).to(device)
ensembled_model.load_state_dict(torch.load(f'{model_path}/model.pt'))
ensembled_model.eval()
ensembled_model.to(device)


### Test model on unbalanced test dset

In [ ]:
metrics = cmc.test_model(model_path, ensembled_model, test_dataloader, max_batch=0)

Processing batches:  57%|█████▋    | 249/436 [00:31<00:24,  7.79it/s]


KeyboardInterrupt: 

### Here the model is tested on individual shots (generates time-confidence graph)

In [ ]:
shots_for_testing = list(map(int,shots_for_testing))
img_path = cmc.per_shot_test(path=model_path, shots=shots_for_testing, results_df=metrics['prediction_df'])

 14%|█▍        | 1/7 [00:00<00:00,  7.15it/s]/compass/Shared/Users/bogdanov/vyzkumny_ukol/confinement_mode_classifier.py:596: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  conf_time_fig, conf_time_ax = plt.subplots(figsize=(10,6))
100%|██████████| 7/7 [00:00<00:00,  7.42it/s]
